# Open Demand

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## Import Database

In [2]:
#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## Define clean_up

In [3]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## Import Tables

In [1]:
# Inventory
inventory_sql = """
SELECT 
    ItemCode
    , WhsCode
    , AVG(OnHand) AS Onhand
FROM 
	SAP.InventoryReportView
WHERE
	WhsCode = 'WW'
	AND OnHand > 0
GROUP BY
	ItemCode
	, WhsCode
"""

# Line Purchase Order
PO_sql = """
WITH PO AS
(SELECT 
	CASE
		WHEN Rate = 0 THEN 1
		ELSE Rate
	END AS EXCHANGE
	, ISBN
	, OpenQuantity
	, GrossPrice
	, ShippingCostUSD
	, Whse
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [DocumentStatus]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
FROM [SAP].[LinePurchaseOrderView]
)

SELECT
	[ISBN]
	, OpenQuantity
	, (GrossPrice*EXCHANGE +ShippingCostUSD) AS LandedCost
	, [Whse]
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
	, (CASE
		WHEN [RowDeliveryDate] > GETDATE() THEN 'OnTime'
		ELSE 'Late'
	END) AS DeliveryStatus
FROM 
	PO
WHERE 
	Whse in ('WW', 'TB') 
	AND OpenQuantity > 0
"""
# Line Sales Order

SO

_sql = """
SELECT 
	Isbn
	, OpenQuantity
	, UnitPrice AS SalePrice
	, WarehouseCode
	, DeliveryDate
	, DocumentNumber
	, DocumentName
	, CustomerOrVendorCode
	, CustomerOrVendorName
	, Condition
	, (CASE
			WHEN DeliveryDate > GETDATE() THEN 'OnTime'
			ELSE 'Late'
		END) AS DeliveryStatus
FROM
	[SAP].[SaleOrderReportView]
WHERE OpenQuantity > 0 AND WarehouseCode IN ('WW', 'AW')
"""

In [ ]:
inventory = pd.read_sql(pub_dict, proc_db)
inventory_ww